# Hiding files with attrib.exe
Detects usage of attrib.exe to hide files from users.

## Rule Content
```
- title: Hiding files with attrib.exe
  id: 4281cb20-2994-4580-aa63-c8b86d019934
  status: experimental
  description: Detects usage of attrib.exe to hide files from users.
  author: Sami Ruohonen
  logsource:
    category: process_creation
    product: windows
    service: null
  detection:
    selection:
      Image: '*\attrib.exe'
      CommandLine: '* +h *'
    ini:
      CommandLine: '*\desktop.ini *'
    intel:
      ParentImage: '*\cmd.exe'
      CommandLine: +R +H +S +A \\*.cui
      ParentCommandLine: C:\WINDOWS\system32\\*.bat
    condition: selection and not (ini or intel)
  fields:
  - CommandLine
  - ParentCommandLine
  - User
  tags:
  - attack.defense_evasion
  - attack.persistence
  - attack.t1158
  falsepositives:
  - igfxCUIService.exe hiding *.cui files via .bat script (attrib.exe a child of cmd.exe
    and igfxCUIService.exe is the parent of the cmd.exe)
  - msiexec.exe hiding desktop.ini
  level: low

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='((process_path.keyword:*\\attrib.exe AND process_command_line.keyword:*\ \+h\ *) AND (NOT ((process_command_line.keyword:*\\desktop.ini\ * OR (process_parent_path.keyword:*\\cmd.exe AND process_command_line.keyword:\+R\ \+H\ \+S\ \+A\ \\*.cui AND process_parent_command_line.keyword:C\:\\WINDOWS\\system32\\*.bat)))))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()